# Processing Gowalla Checkins

This notebook processes the [original Gowalla checkins data](https://snap.stanford.edu/data/loc-gowalla.html) into a database-style format, with a few normalised tables (augmented with some freely-available data).

It represents the first step in a wider project, to create a pipeline from a PostgreSQL database to BigQuery, that automatically triggers updates and displays some simple dashboards in Looker.

---

## Setup

Import packages and other basic setup

In [1]:
## Packages and modules

# standard library
from os import path
from time import sleep

# third-party libs
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
from randomuser import RandomUser

In [40]:
## Constants

# path to raw Gowalla data file
RAW_DATA = path.join("raw", "loc-gowalla_totalCheckins.txt.gz")
# path to processed data files
PROCESSED_DATA = "processed"
# number of checkins to include in sample
SAMPLE_SIZE = 2000

---

## Read data from file, and sample

Pandas has no issues reading the data from a .gz archive, so can just load the file directly. However, a couple of gotchas:
- There are no headers in the file, just the raw data;
- The delimiters used are tabs (i.e. `\t`), not commas.

In [3]:
df = pd.read_csv(
    RAW_DATA,
    header=None,
    names=["user", "check_in_time", "latitude", "longitude", "location_id"],
    delimiter="\t"
)
df.head()

,user,check_in_time,latitude,longitude,location_id
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878


That's what we're after. Let's check a few stats...

In [4]:
print(f"Length of df: {len(df)}")

Length of df: 6442892


In [5]:
for col in ["user", "location_id"]:
    print(f"Number of unique vals in '{col}': {len(df[col].unique())}")

Number of unique vals in 'user': 107092
Number of unique vals in 'location_id': 1280969


In [6]:
107092/6442892

0.016621728254951347

So, there are relatively few unique users, but quite a lot of unique locations. Not good or bad, exactly - just notable.

---

Now, get a randomly-selected sample of these rows (we don't need / want to work with all 6.4m rows for this). 

To reduce the overhead a bit when it comes to building the db, we'll get the number of times each user appears, add this to the original df, and weight the users by this value (so that users with more checkins are more likely to appear in the table)

In [7]:
df_user_counts = df["user"].value_counts().rename_axis("user").reset_index(name="counts")
df_user_counts.head()

,user,counts
0,10971,2175
1,776,2175
2,18931,2150
3,49918,2125
4,620,2125


In [8]:
df = df.merge(df_user_counts, on="user")
df.head()

,user,check_in_time,latitude,longitude,location_id,counts
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847,225
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315,225
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637,225
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516,225
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878,225


In [9]:
df_sample = df.sample(
    n= SAMPLE_SIZE, 
    random_state=1, 
    ignore_index=True,
    weights="counts"
)
df_sample.head()

,user,check_in_time,latitude,longitude,location_id,counts
0,5479,2010-10-03T16:10:09Z,51.924049,4.470105,332428,1693
1,39612,2010-02-21T20:36:02Z,40.763861,-73.972932,12535,1825
2,2,2010-09-21T02:33:01Z,34.089709,-118.268309,167337,2100
3,3384,2010-09-04T20:13:41Z,47.667950,-122.313285,1049172,1950
4,1201,2010-10-20T20:08:59Z,30.274481,-97.739068,898204,1825


Looks good. Now, a quick bit of checking...

In [10]:
for col in df_sample.columns.values:
    print(f"Number of NaN vals in '{col}': {df_sample[col].isna().sum()}")

Number of NaN vals in 'user': 0
Number of NaN vals in 'check_in_time': 0
Number of NaN vals in 'latitude': 0
Number of NaN vals in 'longitude': 0
Number of NaN vals in 'location_id': 0
Number of NaN vals in 'counts': 0


In [11]:
for col in ["user", "location_id"]:
    print(f"Number of unique vals in '{col}': {len(df_sample[col].unique())}")

Number of unique vals in 'user': 1366
Number of unique vals in 'location_id': 1969


So, there are a little over half as many users as there are rows, and nearly all the locations are unique (not ideal, but the level of effort to improve this isn't worth the gain for a relatively quick test)

---

## Enrich and normalise data

The next step is to add a bit of richness: 
- to the user IDs, by creating a set of dummy users that will be in their own table;
- to the locations, by getting some info about the actual physical locations that they correspond to.

We'll create dummy users with the [randomuser package](https://pypi.org/project/randomuser/), which in turn uses the [randomuser.me](https://randomuser.me/) API. We'll get the geo information using [GeoPy](https://geopy.readthedocs.io/en/stable/), which provides an interface to a series of services like Google Maps, Bing Maps and OpenStreetMap Nominatim.

In [12]:
user_list = RandomUser.generate_users(len(df_sample["user"].unique()))

RandomUser contains a series of getter methods, to retrieve the info for each object. Here's an example:

In [13]:
user_list[1].get_nat()

'NO'

We want a dataframe of various properties from each object. Two very obvious ways of constructing this:
- When the pd.DataFrame constructor is called, using a series of list comprehensions (iterate through the list of objects multiple times)
- Iterate through the list of users once, create a series of lists of the properties required, then feed these to the df constructor.

Tbh, it probably makes little difference with such a short list- however, have gone for the 2nd option.

In [14]:
first_names, last_names, dob, nationality, gender, email = [], [], [], [], [], []
for user in user_list:
    first_names.append(user.get_first_name()),
    last_names.append(user.get_last_name()),
    dob.append(user.get_dob()),
    nationality.append(user.get_nat()),
    gender.append(user.get_gender()),
    email.append(user.get_email())

users_df = pd.DataFrame({
    "id": df_sample["user"].unique().tolist(),
    "first_name": first_names,
    "last_name": last_names,
    "dob": dob,
    "nationality": nationality,
    "gender": gender,
    "email": email
})

users_df.head()

,id,first_name,last_name,dob,nationality,gender,email
0,5479,Flavie,Marie,1987-05-15T04:40:11.787Z,FR,female,flavie.marie@example.com
1,39612,Torgeir,Granli,1961-12-17T01:23:44.691Z,NO,male,torgeir.granli@example.com
2,2,Thomas,Hart,1981-08-25T16:07:33.619Z,US,male,thomas.hart@example.com
3,3384,Ouail,Van Velthoven,1962-04-15T04:13:22.109Z,NL,male,ouail.vanvelthoven@example.com
4,1201,Blake,Ramirez,1975-02-16T20:03:54.801Z,GB,male,blake.ramirez@example.com


Perfect (and super-fast too).  Now, to try the geo API, which will _not_ be as quick...

In [15]:
geolocator = Nominatim(user_agent="gcp-testing")

In [16]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1) 

In [19]:
lat_lons = df_sample[["latitude","longitude"]].to_dict("records")
locs = []
for i, row in enumerate(lat_lons):
    try: 
        locs.append(geolocator.reverse((row["latitude"], row["longitude"])))
        if i % 100 == 0:
            print(f"Row {i} complete")
        sleep(1)
    except Exception as err:
        print(f"{err}, row {i}")

Row 0 complete
Row 100 complete
Row 200 complete
Row 300 complete
Row 400 complete
Row 500 complete
Row 600 complete
Row 700 complete
Row 800 complete
Row 900 complete
Row 1000 complete
Row 1100 complete
Row 1200 complete
Row 1300 complete
Row 1400 complete
Row 1500 complete
Row 1600 complete
Row 1700 complete
Row 1800 complete
Row 1900 complete


In [20]:
locs[0].raw

{'place_id': 298442772,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 6923245,
 'lat': '51.9241229',
 'lon': '4.4698094008364455',
 'display_name': 'CS-Kwartier, Centrum, Rotterdam, Zuid-Holland, Nederland',
 'address': {'neighbourhood': 'CS-Kwartier',
  'suburb': 'Centrum',
  'city': 'Rotterdam',
  'municipality': 'Rotterdam',
  'state': 'Zuid-Holland',
  'ISO3166-2-lvl4': 'NL-ZH',
  'country': 'Nederland',
  'country_code': 'nl'},
 'boundingbox': ['51.9237209', '51.9245275', '4.4691519', '4.4704235']}

In [23]:
locs[0].raw["address"]["city"]

'Rotterdam'

In [51]:
country, country_code = [], []
for loc in locs:
    try:
        country.append(loc.raw["address"]["country"])
    except Exception as err:
        if loc.raw["address"]["country_code"] == "pl":
            country.append("Poland")
        else:
            print(f"No country. Raw address:\n{loc.raw['address']}")            
    country_code.append(loc.raw["address"]["country_code"])
    
locations_df = pd.DataFrame({
    "location_id": df_sample["location_id"].tolist(),
    "latitude": df_sample["latitude"].tolist(),
    "longitude": df_sample["longitude"].tolist(),
    "country": country, 
    "country_code": country_code
})

locations_df.head()

,location_id,latitude,longitude,country,country_code
0,332428,51.924049,4.470105,Nederland,nl
1,12535,40.763861,-73.972932,United States,us
2,167337,34.089709,-118.268309,United States,us
3,1049172,47.667950,-122.313285,United States,us
4,898204,30.274481,-97.739068,United States,us


In [52]:
len(locations_df)

2000

In [53]:
locations_df = locations_df.drop_duplicates()

In [54]:
len(locations_df)

1969

In [58]:
locations_df = locations_df.rename(columns={"location_id": "id"})
locations_df.to_csv(path.join(PROCESSED_DATA, "locations.csv"), index=False)

users_df.to_csv(path.join(PROCESSED_DATA, "users.csv"), index=False)

df_sample = df_sample.rename(columns={"user": "user_id"})
df_sample[["user_id", "location_id", "check_in_time"]].to_csv(path.join(PROCESSED_DATA, "checkins.csv"), index=False)

In [59]:
df_sample.head()

,user_id,check_in_time,latitude,longitude,location_id,counts
0,5479,2010-10-03T16:10:09Z,51.924049,4.470105,332428,1693
1,39612,2010-02-21T20:36:02Z,40.763861,-73.972932,12535,1825
2,2,2010-09-21T02:33:01Z,34.089709,-118.268309,167337,2100
3,3384,2010-09-04T20:13:41Z,47.667950,-122.313285,1049172,1950
4,1201,2010-10-20T20:08:59Z,30.274481,-97.739068,898204,1825


In [60]:
locations_df.head()

,id,latitude,longitude,country,country_code
0,332428,51.924049,4.470105,Nederland,nl
1,12535,40.763861,-73.972932,United States,us
2,167337,34.089709,-118.268309,United States,us
3,1049172,47.667950,-122.313285,United States,us
4,898204,30.274481,-97.739068,United States,us


In [61]:
users_df.head()

,id,first_name,last_name,dob,nationality,gender,email
0,5479,Flavie,Marie,1987-05-15T04:40:11.787Z,FR,female,flavie.marie@example.com
1,39612,Torgeir,Granli,1961-12-17T01:23:44.691Z,NO,male,torgeir.granli@example.com
2,2,Thomas,Hart,1981-08-25T16:07:33.619Z,US,male,thomas.hart@example.com
3,3384,Ouail,Van Velthoven,1962-04-15T04:13:22.109Z,NL,male,ouail.vanvelthoven@example.com
4,1201,Blake,Ramirez,1975-02-16T20:03:54.801Z,GB,male,blake.ramirez@example.com
